In [ ]:

#folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
#folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

import os

def find_complement_intersection_names(folder1, folder2):
  """
  This function finds the complement of the intersection of filenames (excluding extensions) in two folders.

  Args:
      folder1 (str): Path to the first folder.
      folder2 (str): Path to the second folder.

  Returns:
      set: Set of filenames (without extensions) that are in one folder but not the other (complement of intersection).
  """
  all_filenames = set(os.path.splitext(f)[0] for f in os.listdir(folder1)) | set(os.path.splitext(f)[0] for f in os.listdir(folder2))
  intersection = all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder1))) & all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder2)))
  complement = all_filenames - intersection

  return complement

# Example usage
folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

complement_names = find_complement_intersection_names(folder1, folder2)

if complement_names:
  print("Complement of intersection (filenames without extensions in one folder but not the other):")
  for filename in complement_names:
    print(filename)
else:
  print("No filenames found in one folder but not the other")

print("Finished checking files.")


 

No filenames found in one folder but not the other
Finished checking files.


In [ ]:
import os
os.chdir("C:/Users/HP14/Desktop/Electronics")


In [1]:
from ultralytics import YOLO
import cv2
import math 
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("BestModel.pt")

# object classes
classNames = ["gun","knife"]


while True:
    success, img = cap.read()
    results = model(img, stream=True)

    #coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            #bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])
            
     
            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 2
            color=(0,0,0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 511.4ms
Speed: 19.5ms preprocess, 511.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 503.4ms
Speed: 16.2ms preprocess, 503.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 472.5ms
Speed: 5.0ms preprocess, 472.5ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 432.1ms
Speed: 4.1ms preprocess, 432.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 334.3ms
Speed: 4.5ms preprocess, 334.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 386.8ms
Speed: 3.1ms preprocess, 386.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 360.2ms
Speed: 6.1ms preprocess, 360.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 376.8ms
Speed: 4.4ms pre

KeyboardInterrupt: 

In [2]:
from ultralytics import YOLO
import cv2
import math
import time
import os
from datetime import datetime

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("BestModel.pt")

# object classes
classNames = ["gun", "knife"]

# confidence threshold
confidence_threshold = 0.5

# specify the directory to save screenshots
screenshot_directory = "screenshots/"

# create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # if confidence is above the threshold, save a screenshot
            if confidence > confidence_threshold:
                current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                screenshot_filename = f"screenshot_{classNames[cls]}_{current_time}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

            # prepare text for display (class name + confidence)
            label = f"{classNames[cls]}: {confidence:.2f}"

            # object details for text placement
            org = [x1, y1 - 10]  # position text above the bounding box
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.9
            color = (255, 255, 255)  # white text
            thickness = 2

            # put the label (class name + confidence) on the image
            cv2.putText(img, label, org, font, fontScale, color, thickness)

    # display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 344.5ms
Speed: 7.8ms preprocess, 344.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 358.5ms
Speed: 6.1ms preprocess, 358.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 383.5ms
Speed: 3.4ms preprocess, 383.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 322.3ms
Speed: 5.3ms preprocess, 322.3ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 301.5ms
Speed: 6.1ms preprocess, 301.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 283.1ms
Speed: 3.5ms preprocess, 283.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 267.4ms
Speed: 3.5ms preprocess, 267.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 282.8ms
Speed: 4.0ms prepr

KeyboardInterrupt: 

In [1]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "http://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun":
        return 0 
    else:
        return 1

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            print(f"Class name --> {class_name}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                    "WeaponType": binary_type,  # 0 for gun, 1 for knife
                    "Confidence": confidence,
                    "Timestamp": current_time
                }

                # Send data to API
                try:
                    headers = {'Content-Type': 'application/json'}
                    response = requests.post(api_url, json=data, headers=headers)
                    if response.status_code == 201:  # 201 is the status code for "Created"
                        print("Data sent to API successfully")
                    else:
                        print(f"Failed to send data to API. Status code: {response.status_code}")
                        print(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    print(f"Error sending data to API: {e}")

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 308.5ms
Speed: 15.4ms preprocess, 308.5ms inference, 11.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 262.9ms
Speed: 6.5ms preprocess, 262.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 211.0ms
Speed: 2.5ms preprocess, 211.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 211.2ms
Speed: 4.1ms preprocess, 211.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 228.4ms
Speed: 2.3ms preprocess, 228.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.3ms
Speed: 3.1ms preprocess, 202.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 207.7ms
Speed: 2.7ms preprocess, 207.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 193.2ms
Speed: 3.1ms pre

KeyboardInterrupt: 

: 

In [3]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json
import logging
import urllib3

# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "https://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun": 
        return False
    else:
        return True

while True:
    success, img = cap.read()
    if not success:
        logging.error("Failed to capture image from webcam")
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            logging.info(f"Confidence: {confidence}")

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            logging.info(f"Class name: {class_name}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                logging.info(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                "detection": {
                "WeaponType": get_binary_object_type,
                "Confidence": float(confidence),
                "Timestamp": current_time
                }
}

                # Send data to API
                try:
                    headers = {'Content-Type': 'application/json'}
                    logging.info(f"Sending data to API: {json.dumps(data, indent=2)}")
                    response = requests.post(api_url, json=data, headers=headers, verify=False)  # Added verify=False
                    logging.info(f"Status Code: {response.status_code}")
                    logging.info(f"Response Body: {response.text}")
                    logging.info(f"Response Headers: {response.headers}")
                    
                    if response.status_code == 201:  # 201 is the status code for "Created"
                        logging.info("Data sent to API successfully")
                    else:
                        logging.error(f"Failed to send data to API. Status code: {response.status_code}")
                        logging.error(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    logging.error(f"Error sending data to API: {e}")
                except Exception as e:
                    logging.error(f"Unexpected error: {e}")

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 213.2ms
Speed: 5.7ms preprocess, 213.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 241.0ms
Speed: 3.3ms preprocess, 241.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 192.6ms
Speed: 3.3ms preprocess, 192.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 225.1ms
Speed: 5.3ms preprocess, 225.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 185.5ms
Speed: 2.2ms preprocess, 185.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 187.5ms
Speed: 2.3ms preprocess, 187.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 212.3ms
Speed: 2.4ms preprocess, 212.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 190.0ms
Speed: 3.0ms prepr

2024-09-19 15:31:50,826 - INFO - Confidence: 0.32
2024-09-19 15:31:50,827 - INFO - Class name: gun


Speed: 3.0ms preprocess, 196.5ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 212.9ms
Speed: 2.4ms preprocess, 212.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 216.3ms
Speed: 4.5ms preprocess, 216.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 200.8ms
Speed: 4.9ms preprocess, 200.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 195.5ms
Speed: 4.1ms preprocess, 195.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 197.4ms
Speed: 4.4ms preprocess, 197.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 190.2ms
Speed: 3.0ms preprocess, 190.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 256.4ms
Speed: 5.7ms preprocess, 256.4ms inference, 0.0ms postp

2024-09-19 15:32:11,823 - INFO - Confidence: 0.3
2024-09-19 15:32:11,824 - INFO - Class name: gun


Speed: 4.1ms preprocess, 190.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 230.6ms
Speed: 3.4ms preprocess, 230.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 208.0ms
Speed: 5.7ms preprocess, 208.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 198.4ms
Speed: 2.5ms preprocess, 198.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 225.7ms
Speed: 3.7ms preprocess, 225.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.8ms
Speed: 2.5ms preprocess, 202.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 221.2ms
Speed: 4.8ms preprocess, 221.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 200.2ms
Speed: 4.1ms preprocess, 200.2ms inference, 1.0ms postp

...

2024-09-19 15:32:22,205 - INFO - Confidence: 0.58
2024-09-19 15:32:22,205 - INFO - Class name: knife
2024-09-19 15:32:22,217 - INFO - Screenshot saved as screenshots/screenshot_knife_2024-09-19T15-32-22.207092.jpg
2024-09-19 15:32:22,218 - INFO - Sending data to API: {
  "WeaponType": 1,
  "Confidence": 0.58,
  "Timestamp": "2024-09-19T15:32:22.207092"
}
2024-09-19 15:32:22,414 - INFO - Status Code: 400
2024-09-19 15:32:22,415 - INFO - Response Body: {"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"errors":{"detection":["The detection field is required."],"$.WeaponType":["The JSON value could not be converted to System.Boolean. Path: $.WeaponType | LineNumber: 0 | BytePositionInLine: 16."]},"traceId":"00-b8d17e19f69b4a3c048a4c859bd54388-9f5cff77fec42501-00"}
2024-09-19 15:32:22,415 - INFO - Response Headers: {'Content-Type': 'application/problem+json; charset=utf-8', 'Date': 'Thu, 19 Sep 2024 12:32:21 GMT', 'Se

Speed: 3.3ms preprocess, 239.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 knife, 216.3ms


2024-09-19 15:32:22,663 - INFO - Confidence: 0.65
2024-09-19 15:32:22,665 - INFO - Class name: knife
2024-09-19 15:32:22,676 - INFO - Screenshot saved as screenshots/screenshot_knife_2024-09-19T15-32-22.665149.jpg
2024-09-19 15:32:22,677 - INFO - Sending data to API: {
  "WeaponType": 1,
  "Confidence": 0.65,
  "Timestamp": "2024-09-19T15:32:22.665149"
}
2024-09-19 15:32:22,758 - INFO - Status Code: 400
2024-09-19 15:32:22,758 - INFO - Response Body: {"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"errors":{"detection":["The detection field is required."],"$.WeaponType":["The JSON value could not be converted to System.Boolean. Path: $.WeaponType | LineNumber: 0 | BytePositionInLine: 16."]},"traceId":"00-88bca5f8767d49ac3b0e90a5ade18417-d4354ba5b6bcaddb-00"}
2024-09-19 15:32:22,759 - INFO - Response Headers: {'Content-Type': 'application/problem+json; charset=utf-8', 'Date': 'Thu, 19 Sep 2024 12:32:22 GMT', 'Se

Speed: 5.4ms preprocess, 216.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 237.9ms
Speed: 5.8ms preprocess, 237.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 258.1ms
Speed: 5.3ms preprocess, 258.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 224.6ms
Speed: 4.5ms preprocess, 224.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 230.6ms
Speed: 3.2ms preprocess, 230.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 211.0ms
Speed: 3.0ms preprocess, 211.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 203.1ms
Speed: 2.9ms preprocess, 203.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 219.8ms
Speed: 2.8ms preprocess, 219.8ms inference, 1.1ms postp

KeyboardInterrupt: 